In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_10_Medoid_Cluster_4_Pvalue_01.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))


In [4]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'


In [5]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp



In [6]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)


In [7]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:

    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"
    
file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_10_Medoid_Cluster_4_Pvalue_01.txt', 'w')
file.write(pos_result)
file.close()


In [8]:

listOfPosResult = pos_result.split('\n\n')
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

98
187
Percentage of profiting pairs in Cluster: 52.406417112299465%


In [9]:
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))

ACN CTXS 0.67
